## Problem 7

In statstics there is a concept called the [Inverse Transform Sampling](https://en.wikipedia.org/wiki/Inverse_transform_sampling) that allows you to generate random numbers from any bijectiv distribution if you have a random number generator that generates numbers from a uniform distribution between 0 and 1. 

The idea behind is based on the fact that the cumulative distribution function of a random variable is monotonic and always between 0 and 1.
In other words, if you have a random variable X with a cumulative distribution function F(x), the cumulative distribution is indeed another random variable Y. 
By calculate the inverse of the cumulative distribution function $F^{-1}$, you can generate random numbers from the distribution of X, by using: $F(Y)^{-1}=X$, with Y being a random number between 0 and 1. 

The principle is demonstrated in the following pictures. 
The first picture shows the general idea of the inverse transform sampling, the second picture shows an example of how to generate random numbers from a normal distribution using the inverse transform sampling.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Generalized_inversion_method.svg/720px-Generalized_inversion_method.svg.png" style="width:400px;"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Inverse_Transform_Sampling_Example.gif" style="width:400px;"/>

To use this sampling method we need a uniform random number generator to generate random numbers between 0 and 1. 
Normally we would use the random number generator of the programming language, but for the sake of this exercise we want to implement our own uniform random number generator using a Linear Congruential Generator [LCG](https://de.wikipedia.org/wiki/Kongruenzgenerator).

This algorithm yields a sequence of pseudo-randomized numbers calculated with a discontinuous piecewise linear equation. 
The method represents one of the oldest and best-known pseudorandom number generator algorithms. 
The theory behind them is relatively easy to understand, and they are easily implemented and fast:
The generator is defined by the recurrence relation:
$$
X_{n+1} = (a X_n + c) \mod m
$$
where: $X_0$ is the seed, and $a$, $c$, and $m$ are constants that define the generator.
The normalization of $X_{n+1}$ with m will result in a Uniform distribution between 0 and 1.

The configuration of $a$, $c$ and $m$ is crucial to the quality of the generator, but there are no rule of thumbs to find the best values.
For example a = 24298, c = 99991, m = 199017 are values that are used in the LCG of the Texas Instruments TI-59 calculator, thus accept that these values are found out by trial and error.
The generated values if plotted in 3D will look like this:

![](https://upload.wikimedia.org/wikipedia/commons/1/1d/TI59_3d.jpg)

Using this algorithm we can create our own uniform random number generator. 

**Your task** is done in 2-steps:
1. Create a class called `BaseGenerator` that implements the following methods:
   - `__init__(self,seed)` to set a `seed`
   - `next(self)` generates the next number in the sequence, but in our implementation raises a `NotImplementedError`
     - `generate(self, n)` generates the next `n` numbers in the sequence and returns them as a `list`
2. Afterwards create a random number generator class called `UniformGenerator`, which inherents from `BaseGenerator` and implements the LCG algorithm, implement the following methods:
   - `def __init__(self, low, high, *, a=1664525, c=1013904223, m=2**32, seed=123)`, where low and high are the boundaries of the uniform distribution and a,c,m, seed are there to init your LCG
   - overwrite `next(self)` and implement LCG
  
3. Create a class ![`ExponentialRandomNumberGenerator`](https://en.wikipedia.org/wiki/Exponential_distribution) that generates random numbers from an exponential distribution using the inverse transform sampling. T
   - `__init__(self, rate, *, a=1664525, c=1013904223, m=2**32, seed=123)` to set the rate of the exponential distribution and the LCG parameters
   - implement `next(self)` that generates the next random number from the exponential distribution

**Bonus** (no points): Create a plot that shows the distribution of the random numbers generated by your `UniformGenerator` and `ExponentialRandomNumberGenerator` class.

In [ ]:
import math

class BaseGenerator:
    def __init__(self, seed):
        self.seed = seed

    def next(self):
        raise NotImplementedError

    def generate(self, n):
        return [self.next() for _ in range(n)]


class UniformRandomNumberGenerator(BaseGenerator):
    def __init__(self, low, high, *, a=1664525, c=1013904223, m=2**32, seed=123):
        super().__init__(seed)
        # normalization with modulus of generator
        self.low = low
        self.high = high

        self.multiplier = a
        self.increment = c
        self.modulus = m

    def next(self):
        new_seed = (self.multiplier * self.seed + self.increment) % self.modulus
        self.seed = new_seed

        # shrink to [0, 1]
        random_number_between_0_and_1 = new_seed / self.modulus
        # scale to [low, high]
        uniform_number = self.low + (self.high - self.low) * random_number_between_0_and_1
        return uniform_number


class ExponentialRandomNumberGenerator(BaseGenerator):
    def __init__(self, rate, *, a=1664525, c=1013904223, m=2**32, seed=123):
        super().__init__(seed)
        self.rate = rate
        self.uniform_generator = UniformRandomNumberGenerator(0, 1, a=a, c=c, m=m, seed=seed)

    def next(self):
        # inverse transform sampling
        # F(x) = 1 - e^(-lambda * x)
        # x = F_inv(y) <-->  x = -ln(1 - y) / lambda
        # y is our uniform variable
        return -math.log(1 - self.uniform_generator.next()) / self.rate

# PROBLEM-TEST
seed = 200
rate = 0.5
low = 1
high = 10
rate = 0.4
generate_sample = 50

uniform_generator = UniformRandomNumberGenerator(low, high, seed=seed)
exponential_generator = ExponentialRandomNumberGenerator(rate, seed=seed)
uniform_generator.generate(generate_sample), exponential_generator.generate(generate_sample)

In [ ]:
# SELF-CHECK
# plot your distribution here

generator_config = {
    'a': 1664525,
    'c': 101390,
    'm': 2**32,
    'seed': 123
}

uniform_generator = UniformRandomNumberGenerator(low = -10, high = 10,**generator_config)
#normal_generator = NormalRandomNumberGenerator(mean = 0, std_dev = 1, **generator_config)
import matplotlib.pyplot as plt
data = uniform_generator.generate(10000)

plt.hist(data,bins=20,edgecolor='k')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()